In [1]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
import re

class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code = True) # trust_remote_code = 모델 다운로드에 대한 검증 절차가 생략
        self.model.to(self.device)

        # 감정매핑
        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """학습된 감정분류 모델 불러오기"""
        self.model.load_state_dict(torch.load(model_file))

    def preprocess_text(self, text):
        """텍스트 전처리"""
        return re.sub("[^0-9a-zA-Z가-힣\s+]", "", text)

    def predict_emotion(self, text):
        """감정 분류 및 예측"""
        # 텍스트 입력 / 토큰화 / 분류
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # 예측하기
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()
        
        # 분류된 감정 라벨
        predicted_emotion = self.label_to_emotion[predicted_label]
        
        return predicted_emotion


In [2]:
import re
import google.generativeai as genai
from IPython.display import display, Markdown
import textwrap
import os
import sys

sys.path.append(os.path.abspath(".."))
from config.api_keys import gemini_key

# Gemini API와 통합
class GeminiService:
    def __init__(self, api_key=gemini_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.0-flash")

    def generate_response(self, emotion, user_text):
        """
        Gemini API를 통해 감정 기반 답변 생성
        """
        prompt = f"너는 사용자가 일기를 쉽게 작성할 수 있도록 도와주고, 유도하는 사람이야. 텍스트에 대해서 자세하게 물어보지 말고 짧게 한줄씩 유도해줘. 사용자의 감정상태는 {emotion} 이고,  사용자의 말: {user_text}"
        
        # 만든 프롬프트를 통해 응답 생성
        response = self.model.generate_content(prompt)
        
        # 응답 텍스트 반환
        return response.text


# 통합 서비스 클래스
class DiaryService:
    def __init__(self, emotion_model_path, gemini_api_key):
        """
        DiaryService 초기화
        :param emotion_model_path: KoBERT 감정 분류 모델 경로
        :param gemini_api_key: Gemini API 키
        """
        # KoBERT 감정 분류 모델 초기화
        self.emotion_classifier = EmotionClassifier()
        self.emotion_classifier.load_model(emotion_model_path)

        # Gemini API 키 저장
        self.gemini_api_key = gemini_api_key

    def process_input(self, user_text):
        """
        사용자 입력 처리 및 응답 생성
        :param user_text: 사용자 입력 텍스트
        :return: 감정 예측 결과 및 Gemini 응답
        """
        # 감정 예측
        predicted_emotion = self.emotion_classifier.predict_emotion(user_text)

        # Gemini API 호출 (예제)
        response = self.get_gemini_response(user_text)

        return {
            "user_text": user_text,
            "predicted_emotion": predicted_emotion,
            "response": response,
        }

    def get_gemini_response(self, text):
        """
        Gemini API 호출 (예제)
        :param text: 사용자 입력 텍스트
        :return: Gemini 응답 텍스트
        """
        # 여기에서 실제 Gemini API 호출 로직을 구현하세요.
        # 예제에서는 간단히 텍스트를 반환합니다.
        return f"Gemini 응답: {text}"

In [3]:
import os
import sys

sys.path.append(os.path.abspath(".."))
from config.api_keys import gemini_key

if __name__ == "__main__":
    # 학습된 KoBERT 모델 파일 경로와 Google Gemini API 키 설정
    emotion_model_path = "new_data_test.pth"

    # 서비스 초기화
    diary_service = DiaryService(emotion_model_path=emotion_model_path, gemini_api_key=gemini_key)

    # 사용자 입력 텍스트 테스트
    user_input = input("텍스트를 입력하세요: ")
    result = diary_service.process_input(user_input)

    print("\n--- 결과 ---")
    print(f"입력 텍스트: {result['user_text']}")
    print(f"예측된 감정: {result['predicted_emotion']}")
    print(f"Gemini 응답: {result['response']}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- 결과 ---
입력 텍스트: 사진에는 사람들이 꽃 구경을 하고있네 다들 웃고있어
예측된 감정: 행복
Gemini 응답: Gemini 응답: 사진에는 사람들이 꽃 구경을 하고있네 다들 웃고있어


In [4]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
import re
import google.generativeai as genai
from datetime import datetime
import os
import sys

sys.path.append(os.path.abspath(".."))
from config.api_keys import gemini_key

# EmotionClassifier: KoBERT 기반 감정 분류 클래스
class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model.to(self.device)

        # 감정 매핑 (라벨 번호 -> 감정 이름)
        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """학습된 모델 불러오기"""
        self.model.load_state_dict(torch.load(model_file))

    def preprocess_text(self, text):
        """텍스트 전처리"""
        return re.sub("[^0-9a-zA-Z가-힣\\s+]", "", text)

    def predict_emotion(self, text):
        """감정 분류 및 예측"""
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}

        # 모델 예측
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()

        # 예측된 감정을 반환
        return self.label_to_emotion[predicted_label]


# Gemini API와 통합
class GeminiService:
    def __init__(self, api_key=gemini_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.0-flash")

    def generate_response(self, emotion, user_text):
        """
        Gemini API를 통해 감정 기반 질문 생성
        """
        prompt = f"너는 사용자가 일기를 쉽게 작성할 수 있도록 도와주고, 유도하는 사람이야. 텍스트에 대해서 자세하게 물어보지 말고 짧게 한줄씩 유도해줘. 사용자의 감정상태는 {emotion} 이고,  사용자의 말: {user_text}"
        
        # Generate content using Gemini API
        response = self.model.generate_content(prompt)
        
        # 응답 텍스트 반환
        return response.text

    def summarize_conversation(self, conversation_history):
        """
        Gemini API를 통해 대화 내용을 요약하여 일기 형식으로 작성
        """
        prompt = f"""
대화내용을 일기형식으로 요약:
{conversation_history}
일기 형식으로 작성해줘야 해
"""
        
        # Generate summary using Gemini API
        response = self.model.generate_content(prompt)
        
        # 요약된 텍스트 반환
        return response.text


# 통합 서비스 클래스
class DiaryService:
    def __init__(self, emotion_model_path, gemini_api_key):
        # KoBERT 기반 감정 분류 모델 초기화
        self.classifier = EmotionClassifier()
        self.classifier.load_model(emotion_model_path)

        # Gemini API 초기화
        self.gemini_service = GeminiService(api_key=gemini_api_key)

    def process_input(self, user_text):
        """
        사용자 입력 처리 및 응답 생성
        """
        # 감정 예측
        predicted_emotion = self.classifier.predict_emotion(user_text)

        # Gemini API로 질문 생성
        response_text = self.gemini_service.generate_response(predicted_emotion, user_text)

        return predicted_emotion, response_text

    def summarize_conversation(self, conversation_history):
        """
        대화 내용 요약 요청
        """
        return self.gemini_service.summarize_conversation(conversation_history)


# 메인 실행 코드 (대화 인터페이스)
if __name__ == "__main__":
    # 학습된 KoBERT 모델 파일 경로와 Google Gemini API 키 설정
    emotion_model_path = "new_data_test.pth"
    gemini_api_key = gemini_key

    # 서비스 초기화
    diary_service = DiaryService(emotion_model_path=emotion_model_path, gemini_api_key=gemini_api_key)

    print("인용이의 일기 도우미에 오신 것을 환영합니다! '그만'이라고 입력하면 대화를 종료하고 요약된 일기를 보여줍니다.\n")

    conversation_history = ""
    while True:
        # 사용자 입력 받기
        user_input = input("사용자: ")
        
        if user_input.strip().lower() == "그만":
            print("\n대화를 종료하고 요약된 일기를 생성합니다...\n")
            diary_entry = diary_service.summarize_conversation(conversation_history)
            
            # 현재 날짜 및 시간 가져오기
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            print(f"### 일기 작성 ({current_time})\n")
            print(diary_entry)
            break

        # 사용자 입력 처리 및 응답 생성
        predicted_emotion, gemini_response = diary_service.process_input(user_input)

        # 대화 기록 저장
        conversation_history += f"User: {user_input}\n인용이: {gemini_response}\n"

        # AI 응답 출력
        print(f"인용이: {gemini_response}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


인용이의 일기 도우미에 오신 것을 환영합니다! '그만'이라고 입력하면 대화를 종료하고 요약된 일기를 보여줍니다.

인용이: 오늘 어떤 놀라운 일이 있었나요?

인용이: 오늘따라 더 예뻐 보이는 꽃이 있나요?

인용이: 오늘 유채꽃을 보니 어떤 점이 가장 놀라웠나요?

인용이: 오늘 하루를 싱싱하게 만들어준 것은 무엇이었나요?


대화를 종료하고 요약된 일기를 생성합니다...

### 일기 작성 (2025-03-02 13:58:29)

## 오늘 하루 일기

오늘, 누군가가 나를 따라 하는 듯한 느낌을 받았다. 묘한 기분이었다. 그러다 문득, 사람들이 꽃구경하는 모습이 눈에 들어왔다. 다들 웃고 있는 모습이 보기 좋았다. 특히 유채꽃이 눈에 띄었는데, 정말 싱싱하더라. 그 싱싱함 덕분인지, 오늘 하루도 덩달아 싱싱하게 느껴졌다. 유채꽃 덕분에 왠지 모르게 기분 좋은 하루였다.

